# Bedrock AgentCore Memory Store Tutorial

This tutorial demonstrates the complete functionality of the BedrockAgentCoreMemoryStore, which provides persistent memory capabilities for LangGraph agents using Amazon Bedrock AgentCore Memory service.

## Prerequisites

1. AWS credentials configured
2. Bedrock AgentCore Memory resource created and ACTIVE
3. Required packages installed

## Setup and Configuration

In [2]:
import logging
from datetime import datetime
from typing import Annotated

from bedrock_agentcore.memory.client import MemoryClient
from bedrock_agentcore_memory_store import BedrockAgentCoreMemoryStore
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.store.base import BaseStore
from typing_extensions import TypedDict

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Configuration - Update these values
REGION = "us-west-2"
MEMORY_ID = "MEMORY_ID"  # Replace with your actual memory ID

# Initialize memory client and store
memory_client = MemoryClient(REGION)
store = BedrockAgentCoreMemoryStore(memory_client=memory_client, memory_id=MEMORY_ID)

print(f"Memory store initialized with ID: {MEMORY_ID}")
print(f"Supports TTL: {store.supports_ttl}")

INFO:botocore.credentials:Found credentials in environment variables.
INFO:bedrock_agentcore.memory.client:Initialized MemoryClient for control plane: us-west-2, data plane: us-west-2


Memory store initialized with ID: MEMORY_ID
Supports TTL: False


## Core Store Operations

### 1. Storing Messages (put operation)

In [4]:
# Define namespace (actor_id, session_id)
namespace = ("user-123", "session-456")

# Store different types of messages
messages_to_store = [
    ("msg-1", HumanMessage("I love playing soccer and my favorite team is Barcelona")),
    ("msg-2", AIMessage("That's great! Barcelona has a rich history in football.")),
    ("msg-3", HumanMessage("My name is John and I'm a software engineer")),
    ("msg-4", AIMessage("Nice to meet you John! Software engineering is a fascinating field.")),
    ("msg-5", HumanMessage("I prefer Python for backend development")),
    ("msg-6", SystemMessage("User preferences updated: Python, Backend Development")),
]

for key, message in messages_to_store:
    try:
        store.put(namespace, key, {"message": message})
        print(f"✅ Stored {message.type} message with key: {key}")
    except Exception as e:
        print(f"❌ Failed to store message {key}: {e}")

INFO:bedrock_agentcore.memory.client:Created event: 0000001757113343000#0f3ea8da


✅ Stored human message with key: msg-1


INFO:bedrock_agentcore.memory.client:Created event: 0000001757113344000#b3cf6a09
INFO:bedrock_agentcore.memory.client:Created event: 0000001757113345000#866aac05


✅ Stored ai message with key: msg-2
✅ Stored human message with key: msg-3


INFO:bedrock_agentcore.memory.client:Created event: 0000001757113345000#abb0945d


✅ Stored ai message with key: msg-4


ERROR:bedrock_agentcore.memory.client:Failed to create event: An error occurred (ThrottledException) when calling the CreateEvent operation (reached max retries: 4): Rate exceeded.


❌ Failed to store message msg-5: An error occurred (ThrottledException) when calling the CreateEvent operation (reached max retries: 4): Rate exceeded.


INFO:bedrock_agentcore.memory.client:Created event: 0000001757113356000#175223c3


✅ Stored system message with key: msg-6


### 2. Searching and Retrieving Memories

Wait for messages to be processed by AgentCore (typically 30-60 seconds)

In [ ]:
import time

print("Waiting 45 seconds for AgentCore to process messages...")
time.sleep(45)
print("Processing complete!")

In [6]:
# Search namespace for processed memories
search_namespace = ("summaries", "user-123", "session-456")

# Basic listing (no semantic search)
print("=== Basic Listing ===")
try:
    results = store.search(search_namespace, limit=10)
    print(f"Found {len(results)} memory records")
    
    for i, item in enumerate(results[:3]):
        print(f"\nRecord {i+1}:")
        print(f"  Key: {item.key}")
        print(f"  Content: {item.value.get('content', '')[:100]}...")
        print(f"  Created: {item.created_at}")
except Exception as e:
    print(f"❌ Search failed: {e}")

=== Basic Listing ===
Found 1 memory records

Record 1:
  Key: mem-e2facd2a22ded39a2e94e522b857bfaba224
  Content: <summary>
    <topic name="User Information">
        The user&#39;s name is John, and he works as a...
  Created: 2025-09-05 16:02:36-07:00


In [7]:
# Semantic search with queries
print("=== Semantic Search ===")

search_queries = [
    "soccer and Barcelona",
    "user preferences and programming",
    "John software engineer",
    "Python backend development"
]

for query in search_queries:
    try:
        results = store.search(
            search_namespace,
            query=query,
            limit=3
        )
        
        print(f"\nQuery: '{query}'")
        print(f"Results: {len(results)}")
        
        for item in results:
            score = getattr(item, 'score', 'N/A')
            content = item.value.get('content', '')[:80]
            print(f"  Score: {score} | Content: {content}...")
            
    except Exception as e:
        print(f"❌ Search failed for '{query}': {e}")

=== Semantic Search ===


INFO:bedrock_agentcore.memory.client:Retrieved 1 memories from namespace: /summaries/user-123/session-456



Query: 'soccer and Barcelona'
Results: 1
  Score: 0.42368466 | Content: <summary>
    <topic name="User Information">
        The user&#39;s name is Joh...


INFO:bedrock_agentcore.memory.client:Retrieved 1 memories from namespace: /summaries/user-123/session-456



Query: 'user preferences and programming'
Results: 1
  Score: 0.43479428 | Content: <summary>
    <topic name="User Information">
        The user&#39;s name is Joh...


INFO:bedrock_agentcore.memory.client:Retrieved 1 memories from namespace: /summaries/user-123/session-456



Query: 'John software engineer'
Results: 1
  Score: 0.4470298 | Content: <summary>
    <topic name="User Information">
        The user&#39;s name is Joh...


INFO:bedrock_agentcore.memory.client:Retrieved 1 memories from namespace: /summaries/user-123/session-456



Query: 'Python backend development'
Results: 1
  Score: 0.43134356 | Content: <summary>
    <topic name="User Information">
        The user&#39;s name is Joh...


### 3. Get Individual Memory Records

In [9]:
# Get a specific memory record by ID
print("=== Get Individual Records ===")

# First get some record IDs from search
try:
    search_results = store.search(search_namespace, limit=2)
    
    if search_results:
        record_id = search_results[0].key
        print(f"Testing get operation with record ID: {record_id}")
        
        # Get the specific record
        result = store.get(namespace, record_id)
        
        if result:
            print(f"✅ Retrieved record:")
            print(f"  Key: {result.key}")
            print(f"  Namespace: {result.namespace}")
            print(f"  Content: {result.value.get('content', '')}")
            print(f"  Created: {result.created_at}")
        else:
            print("❌ No record returned")
    else:
        print("No search results available for get test")
        
except Exception as e:
    print(f"❌ Get operation failed: {e}")

# Test get with non-existent key
try:
    result = store.get(namespace, "non-existent-key")
    if result is None:
        print("✅ Correctly returned None for non-existent key")
    else:
        print("❌ Should have returned None")
except Exception as e:
    print(f"❌ Get non-existent failed: {e}")

=== Get Individual Records ===
Testing get operation with record ID: mem-e2facd2a22ded39a2e94e522b857bfaba224
✅ Retrieved record:
  Key: mem-e2facd2a22ded39a2e94e522b857bfaba224
  Namespace: ('', 'summaries', 'user-123', 'session-456')
  Content: <summary>
    <topic name="User Information">
        The user&#39;s name is John, and he works as a software engineer. He enjoys playing soccer and supports Barcelona football club.
    </topic>
    <topic name="Conversation">
        The assistant acknowledged John&#39;s interest in Barcelona, noting the team&#39;s rich history in football. The assistant also greeted John and commented that software engineering is a fascinating field.
    </topic>
    <topic name="User Preferences">
        User preferences were updated to include Python and Backend Development.
    </topic>
</summary>
  Created: 2025-09-05 16:02:36-07:00
❌ Get non-existent failed: An error occurred (ResourceNotFoundException) when calling the GetMemoryRecord operation: Reso

### 4. Delete Operations

In [12]:
# Test delete operations
print("=== Delete Operations ===")

print(f"Record ID to delete: {record_id}")

try:
    # Delete the record
    store.delete(namespace, record_id)
    print(f"✅ Delete operation called for key: {record_id}")
    
except Exception as e:
    print(f"❌ Delete operation failed: {e}")

=== Delete Operations ===
Record ID to delete: mem-e2facd2a22ded39a2e94e522b857bfaba224
✅ Delete operation called for key: mem-e2facd2a22ded39a2e94e522b857bfaba224


In [16]:
# Verify deletion
try:
    result = store.get(namespace, record_id)
except Exception as e:
    print(f"✅ Record {record_id} was successfully deleted")

✅ Record mem-e2facd2a22ded39a2e94e522b857bfaba224 was successfully deleted


## Complete Agent Example with Memory

This demonstrates how to use the memory store in a complete LangGraph agent.

In [18]:
# Initialize LLM
llm = init_chat_model(
    "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    model_provider="bedrock_converse",
)

# Define agent state
class State(TypedDict):
    messages: Annotated[list, add_messages]

def call_model(state: State, config: RunnableConfig, *, store: BaseStore):
    # Get configuration
    user_id = config["configurable"]["user_id"]
    session_id = config["configurable"]["session_id"]
    
    # Store user message
    conversation_namespace = (user_id, session_id)
    user_message = state['messages'][-1]
    
    print(f"Storing user message: {user_message.content[:50]}...")
    store.put(
        conversation_namespace,
        f"user-msg-{int(time.time())}",
        value={"message": user_message}
    )
    
    # Search for relevant memories
    memory_namespace = ("summaries", user_id, session_id)
    memories = store.search(
        memory_namespace,
        query=user_message.content,
        limit=3
    )
    
    print(f"Found {len(memories)} relevant memories")
    
    # Add memory context to messages if available
    messages = state["messages"].copy()
    if memories:
        memory_context = "\n".join([
            f"Memory: {mem.value.get('content', '')[:100]}..."
            for mem in memories[:2]
        ])
        context_msg = SystemMessage(f"Relevant memories:\n{memory_context}")
        messages.insert(-1, context_msg)
    
    # Generate response
    result = llm.invoke(messages)
    
    # Store AI response
    print(f"Storing AI response: {result.content[:50]}...")
    store.put(
        conversation_namespace,
        f"ai-msg-{int(time.time())}",
        value={"message": result}
    )
    
    return {"messages": [result]}

# Build graph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", call_model)
graph_builder.add_edge(START, "chatbot")

# Compile with memory store
checkpointer = InMemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer, store=store)

print("✅ Agent with memory compiled successfully")

✅ Agent with memory compiled successfully


In [20]:
import time

# Test the agent with memory
print("=== Testing Agent with Memory ===")

# Configuration
config = {
    "configurable": {
        "user_id": "demo-user",
        "session_id": "demo-session",
        "thread_id": "demo-thread"
    }
}

# Test conversations
test_messages = [
    "Hi, I'm interested in learning about machine learning",
    "What programming languages are best for ML?",
    "Can you remind me what we discussed about programming?"
]

for i, message in enumerate(test_messages):
    print(f"\n--- Conversation {i+1} ---")
    print(f"User: {message}")
    
    try:
        result = graph.invoke(
            {"messages": [HumanMessage(message)]},
            config
        )
        
        ai_response = result['messages'][-1].content
        print(f"AI: {ai_response[:200]}...")
        
    except Exception as e:
        print(f"❌ Agent failed: {e}")
    
    # Wait between messages
    time.sleep(2)

INFO:bedrock_agentcore.memory.client:Created event: 0000001757113835000#7e243282


=== Testing Agent with Memory ===

--- Conversation 1 ---
User: Hi, I'm interested in learning about machine learning
Storing user message: Hi, I'm interested in learning about machine learn...


INFO:bedrock_agentcore.memory.client:Retrieved 0 memories from namespace: /summaries/demo-user/demo-session
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Found 0 relevant memories


INFO:bedrock_agentcore.memory.client:Created event: 0000001757113843000#fcf5f078


...


--- Conversation 3 ---
User: Can you remind me what we discussed about programming?
Storing user message: Can you remind me what we discussed about programm...


INFO:bedrock_agentcore.memory.client:Retrieved 0 memories from namespace: /summaries/demo-user/demo-session
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Found 0 relevant memories


INFO:bedrock_agentcore.memory.client:Created event: 0000001757113860000#0a1659ad


Storing AI response: # Our Previous Discussion on Programming for Machi...
AI: # Our Previous Discussion on Programming for Machine Learning

In our conversation, we discussed programming languages that are best suited for machine learning work. Here's a summary of what I shared...


## Key Features Summary

The BedrockAgentCoreMemoryStore provides:

### Core Operations
- **put()**: Store messages with automatic processing by AgentCore
- **search()**: Semantic search and listing of processed memories
- **get()**: Retrieve individual memory records by ID
- **delete()**: Remove memory records
- **batch()**: Execute multiple operations efficiently

### Memory Processing
- Automatic message processing and summarization
- Semantic search capabilities
- User preference extraction
- Long-term memory persistence

### Integration Features
- LangGraph store interface compliance
- Support for all LangChain message types
- Namespace-based organization
- Configurable search parameters

### Limitations
- No async operations support
- TTL managed by AgentCore service
- Custom indexing handled by AgentCore
- Processing delay for memory availability

This memory store enables persistent, searchable memory for conversational AI agents with automatic processing and intelligent retrieval capabilities.